# Deploy model as a webservice on Azure Kubernetes Service

In [1]:
import numpy as np
import azureml.core

# display the core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.33.0


In [2]:
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, sep = '\n')

mlops-workspace
learn_mlops_2
centralus


# Deploy model 

In [3]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
import joblib
import time
from azureml.core.model import Model
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
lemmatizer=WordNetLemmatizer()

def preprocess_data(data):
    corpus=[]
    for i in data:
        mess=re.sub("[^a-zA-Z0-9]"," ",i)
        mess=mess.lower().split()
        mess=[lemmatizer.lemmatize(word) for word in mess if word not in stopwords.words("english")]
        mess=" ".join(mess)
        corpus.append(mess)
    return corpus    


def init():
    global count_vect,rf_model
    
    count_vect_path=Model.get_model_path('NLP_Count_Vectorizer')
    count_vect= joblib.load(count_vect_path)
    
    rf_model_path=Model.get_model_path('NLP_RF_Model')
    rf_model=joblib.load(rf_model_path)
    
def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        corpus=preprocess_data(data[0])
        count_test=count_vect.transform(corpus)
        prediction=rf_model.predict(count_test)
        # you can return any data type as long as it is JSON-serializable
        return json.dumps({"result": prediction.tolist()})
    except Exception as e:
        result = str(e)
        # return error message back to the client
        return json.dumps({"error": result})
        

Overwriting score.py


# Define Environment

In [5]:
from azureml.core.environment import Environment
from azureml.core.environment import CondaDependencies


# Create the environment
myenv = Environment(name="MyEnvironment")

# Create the dependencies object
print("Creating dependencies....")
myenv_dep = CondaDependencies.create(conda_packages=['scikit-learn', 'pip'],
                                     pip_packages=['azureml-defaults','joblib','numpy','azureml-core', "azureml-monitoring", "inference-schema", "inference-schema[numpy-support]","nltk"])

myenv.python.conda_dependencies = myenv_dep

# Register the environment
print("Registering the environment...")
myenv.register(ws)

Creating dependencies....
Registering the environment...


{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210714.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "MyEnvironment",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"


# Deployment Configuration

In [11]:
from azureml.core.webservice import AksWebservice

aks_config = AksWebservice.deploy_configuration(auth_enabled=True)

#  Deploy web service

In [25]:
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.compute import AksCompute, ComputeTarget

# Choose a name for your AKS cluster
aks_name = 'nlp-aks' 

# Verify that cluster does not exist already
try:
    aks_target = ComputeTarget(workspace=ws, name=aks_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # Use the default configuration (you can also provide parameters to customize this).
    # For example, to create a dev/test cluster, use:
    # prov_config = AksCompute.provisioning_configuration(cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)
    prov_config = AksCompute.provisioning_configuration(cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)

    # Create the cluster
    aks_target = ComputeTarget.create(workspace = ws, 
                                    name = aks_name, 
                                    provisioning_configuration = prov_config)

if aks_target.get_status() != "Succeeded":
    aks_target.wait_for_completion(show_output=True)

InProgress..............................................................
SucceededProvisioning operation finished, operation "Succeeded"


In [26]:
model1 = Model(ws, 'NLP_Count_Vectorizer')
model2 = Model(ws, 'NLP_RF_Model')

service_name = 'nlp-sent-analysis'

In [27]:
from azureml.core.model import InferenceConfig
inference_config = InferenceConfig(entry_script='score.py',
                                    environment=myenv)

In [28]:
service = Model.deploy(ws, service_name, models=[model1, model2], inference_config=inference_config, deployment_config=aks_config, deployment_target=aks_target,overwrite=True)
service.wait_for_deployment(show_output = True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-08-22 08:22:10+00:00 Creating Container Registry if not exists.
2021-08-22 08:22:10+00:00 Registering the environment.
2021-08-22 08:22:14+00:00 Use the existing image..
2021-08-22 08:22:16+00:00 Creating resources in AKS.
2021-08-22 08:22:17+00:00 Submitting deployment to compute.
2021-08-22 08:22:17+00:00 Checking the status of deployment nlp-sent-analysis..
2021-08-22 08:24:17+00:00 Checking the status of inference endpoint nlp-sent-analysis.
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy


In [29]:
print(service.get_logs())

2021-08-22T08:24:11,184065623+00:00 - rsyslog/run 
2021-08-22T08:24:11,184066523+00:00 - iot-server/run 
2021-08-22T08:24:11,184066523+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-08-22T08:24:11,269632566+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-08-22T08:24:11,351373884+00:00 - iot-server/finish 1 0
2021-08-22T08:24:11,352499591+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (15)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 42
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2021-08-22 08:24:14,015 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2021-08-22 08:24:14,015 | root | INFO | Starting up request id generator
2021-08-22 08:24:14,015 | root | INFO | Star

In [30]:
service.update(enable_app_insights=True)

# Test web service 

In [31]:
print(service.scoring_uri)

http://40.89.255.53:80/api/v1/service/nlp-sent-analysis/score


In [32]:
print(service.swagger_uri)

http://40.89.255.53:80/api/v1/service/nlp-sent-analysis/swagger.json


In [33]:
service.state

'Healthy'

In [34]:
import json


input_payload = json.dumps({
    'data': [["I love this phone , It is very handy and has a lot of features ."]],
     # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)

{"result": [1]}
